# Querying a SPARQL endpoint (knowledge graph) in python
The code below contains everything needed to execute a SPARQL query, manipulate the data it returns, and store the data in a file suitable for rendering with cytoscape.  Your goal is to get this to work locally and attempt to extend it by, for example, altering the query, sorting the results differently, or working with the output in cytoscape to build an informative visualization.  


In [1]:
import pandas as pd

from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

## Create query function

The function below takes a SPARQL query string, queries a sparql service, and returns the result as a pandas DataFrame (a table).

In [ ]:
def query_wikidata(sparql_query, sparql_service_url):
    """
    Query the endpoint with the given query string and return the results as a pandas Dataframe.
    """
    # create the connection to the endpoint
    sparql = SPARQLWrapper(sparql_service_url)
    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # ask for the result
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

## Run our SPARQL query
The example here is the drug -> interacts with protein -> encoded by -> gene -> gwas -> disease query pattern. 
Test it here http://tinyurl.com/hwm9388 and explore the results to see you you might adapt it.  Other example queries for the wikidata service:  http://tinyurl.com/j222k6g ,  http://tinyurl.com/gpfr9kj 

In [ ]:
sparql_query = """SELECT ?drug ?drugLabel ?gene ?geneLabel ?entrez_id ?disease ?diseaseLabel WHERE {
      ?drug wdt:P129 ?gene_product .   # drug interacts with a gene_product 
      ?gene_product wdt:P702 ?gene .  # gene_product is encoded by a gene
      ?gene wdt:P2293 ?disease .    # gene is genetically associated with a disease 
      ?gene wdt:P351 ?entrez_id .  # get the entrez gene id for the gene 
      # add labels
      SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en" .
      }
    }
    limit 1000
    """
#to query another endpoint, change the URL for the service and the query
sparql_service_url = "https://query.wikidata.org/sparql"
result_table = query_wikidata(sparql_query, sparql_service_url)

<ipython-input-2-3c2b1717a014>:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  return json_normalize(result["results"]["bindings"])


## From here on we are using the Python Data Analysis Library "Pandas"
Look for an introduction like this http://synesthesiam.com/posts/an-introduction-to-pandas.html if you get stuck..

Now look at the results of our SPARQL query.  "shape" shows the dimensions (n rows by n cols).  head() shows the column headers (which will be important later) and a sample of the first few rows of data

In [ ]:
result_table.shape

(1000, 17)

In [ ]:
result_table

,drug.type,drug.value,gene.type,gene.value,disease.type,disease.value,entrez_id.type,entrez_id.value,drugLabel.xml:lang,drugLabel.type,drugLabel.value,geneLabel.xml:lang,geneLabel.type,geneLabel.value,diseaseLabel.xml:lang,diseaseLabel.type,diseaseLabel.value
0,uri,http://www.wikidata.org/entity/Q132621,uri,http://www.wikidata.org/entity/Q17710464,uri,http://www.wikidata.org/entity/Q61335,literal,146,en,literal,epinephrine,en,literal,ADRA1D,en,literal,pre-eclampsia
1,uri,http://www.wikidata.org/entity/Q186242,uri,http://www.wikidata.org/entity/Q17710464,uri,http://www.wikidata.org/entity/Q61335,literal,146,en,literal,norepinephrine,en,literal,ADRA1D,en,literal,pre-eclampsia
2,uri,http://www.wikidata.org/entity/Q221361,uri,http://www.wikidata.org/entity/Q17710464,uri,http://www.wikidata.org/entity/Q61335,literal,146,en,literal,clozapine,en,literal,ADRA1D,en,literal,pre-eclampsia
3,uri,http://www.wikidata.org/entity/Q280786,uri,http://www.wikidata.org/entity/Q17710464,uri,http://www.wikidata.org/entity/Q61335,literal,146,en,literal,terazosin,en,literal,ADRA1D,en,literal,pre-eclampsia
4,uri,http://www.wikidata.org/entity/Q408963,uri,http://www.wikidata.org/entity/Q17710464,uri,http://www.wikidata.org/entity/Q61335,literal,146,en,literal,indoramin,en,literal,ADRA1D,en,literal,pre-eclampsia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,uri,http://www.wikidata.org/entity/Q21172597,uri,http://www.wikidata.org/entity/Q14911688,uri,http://www.wikidata.org/entity/Q827497,literal,4436,en,literal,Inclusion membrane protein CTL0184,en,literal,MSH2,en,literal,Muir-Torre syndrome
996,uri,http://www.wikidata.org/entity/Q21168330,uri,http://www.wikidata.org/entity/Q14911717,uri,http://www.wikidata.org/entity/Q188874,literal,2956,en,literal,Inclusion membrane protein CT_813,en,literal,MSH6,en,literal,colorectal cancer
997,uri,http://www.wikidata.org/entity/Q21172597,uri,http://www.wikidata.org/entity/Q14911717,uri,http://www.wikidata.org/entity/Q188874,literal,2956,en,literal,Inclusion membrane protein CTL0184,en,literal,MSH6,en,literal,colorectal cancer
998,uri,http://www.wikidata.org/entity/Q21168330,uri,http://www.wikidata.org/entity/Q14911717,uri,http://www.wikidata.org/entity/Q736633,literal,2956,en,literal,Inclusion membrane protein CT_813,en,literal,MSH6,en,literal,Mismatch repair cancer syndrome


Compare the column names to the first line of the SPARQL query:

SELECT ?drug ?drugLabel ?gene ?geneLabel ?entrez_id ?disease ?diseaseLabel

Each of the SELECTed items (e.g. ?drug) results in 2 columns: one for its data type (either uri or literal) and one for its value (the thing you are probably looking for).  String literal values for labels have an additional column indicating which language they are in.  

For the purposes of this exercise, we can simplify this to focus only on the string names of the drug, disease and gene in each row in the results.  As follows.

## Extract only the columns we care about

In [ ]:
simple_table = result_table[["drugLabel.value", "diseaseLabel.value", "geneLabel.value"]]

In [ ]:
simple_table.head()

,drugLabel.value,diseaseLabel.value,geneLabel.value
0,epinephrine,pre-eclampsia,ADRA1D
1,norepinephrine,pre-eclampsia,ADRA1D
2,clozapine,pre-eclampsia,ADRA1D
3,terazosin,pre-eclampsia,ADRA1D
4,indoramin,pre-eclampsia,ADRA1D


### Rename the columns of our simple table

Let's delete the "Label.value" portion from the column names.

In [ ]:
simple_table = simple_table.rename(columns = lambda col: col.replace("Label.value", ""))

In [ ]:
simple_table.head()

,drug,disease,gene
0,epinephrine,pre-eclampsia,ADRA1D
1,norepinephrine,pre-eclampsia,ADRA1D
2,clozapine,pre-eclampsia,ADRA1D
3,terazosin,pre-eclampsia,ADRA1D
4,indoramin,pre-eclampsia,ADRA1D


We have just grabbed the three columns we care about and renamed them.

## Count the number of genes per (drug, disease) pair

How many genes link each unique (drug, disease) pair? After counting, order the (drug, disease) pairs in descending order of number of linking genes.  This is one simple method for finding stronger connections between A and C concepts - based on the number of shared B concepts.  

### Make unique (drug, disease) groups and count the number of genes for this group

In [ ]:
counts = simple_table.groupby(["drug", "disease"]).size()

In [ ]:
counts.head()

drug                            disease                                             
(-)-pentazocine                 amyotrophic lateral sclerosis type 16                   1
                                autosomal recessive distal spinal muscular atrophy 2    1
                                coronary artery disease                                 1
(25R)-cholest-5-ene-3β,26-diol  metabolic disease                                       1
(RS)-methadone                  coronary artery disease                                 1
dtype: int64

### Turn result into a Pandas data frame (fancy smart table)

In [ ]:
counts = counts.to_frame("gene_count")

In [ ]:
counts.head()

gene_count
drug                           disease                                                       
(-)-pentazocine                amyotrophic lateral sclerosis type 16                        1
                               autosomal recessive distal spinal muscular atro...           1
                               coronary artery disease                                      1
(25R)-cholest-5-ene-3β,26-diol metabolic disease                                            1
(RS)-methadone                 coronary artery disease                                      1

### Put the drug and disease labels back into columns

In [ ]:
counts = counts.reset_index()

In [ ]:
counts.head()

,drug,disease,gene_count
0,(-)-pentazocine,amyotrophic lateral sclerosis type 16,1
1,(-)-pentazocine,autosomal recessive distal spinal muscular atr...,1
2,(-)-pentazocine,coronary artery disease,1
3,"(25R)-cholest-5-ene-3β,26-diol",metabolic disease,1
4,(RS)-methadone,coronary artery disease,1


### Sort the table in descending order of genes

In [ ]:
counts = counts.sort_values("gene_count", ascending = False)

In [ ]:
counts.head()

,drug,disease,gene_count
192,Nintedanib,Jackson–Weiss syndrome,2
609,linifanib,acute myeloid leukemia,2
888,sunitinib,acute myeloid leukemia,2
383,crenolanib,acute myeloid leukemia,2
351,cediranib,acute myeloid leukemia,2


### Number the results from 0 onwards

In [ ]:
counts = counts.reset_index(drop = True)

In [ ]:
counts.head()

,drug,disease,gene_count
0,Nintedanib,Jackson–Weiss syndrome,2
1,linifanib,acute myeloid leukemia,2
2,sunitinib,acute myeloid leukemia,2
3,crenolanib,acute myeloid leukemia,2
4,cediranib,acute myeloid leukemia,2


---

## Add the genes used to link each (drug, disease) pair

Now add another column containing the actual genes linking each (drug, disease) pair.

### Create a dictionary containing the linking genes for each unique pair

In [ ]:
linking_genes = dict()
for (drug, disease), small_table in simple_table.groupby(["drug", "disease"]):
    linking_genes[(drug, disease)] = list(small_table["gene"])

### Example: retrieve the linking genes for (caffeine, obesity)

In [ ]:
linking_genes[("Caffeine", "obesity")]

KeyError: ('Caffeine', 'obesity')

### Make a new column containing the linking genes

In [ ]:
counts["genes"] = counts[["drug", "disease"]].apply(
    lambda row: linking_genes[(row["drug"], row["disease"])],
    axis = 1
)

In [ ]:
counts.head()

## Save to file

In [ ]:
counts.to_csv("drug_disease_count.tsv", sep = '\t', index = False, encoding = 'utf-8')

---

## Make cytoscape file

For the example query, the sorting based on shared gene count seems unsatisfying.  Now, lets look at all the results in a network view to see if any interesting patterns emerge that might shed some light on the data and how we might process it more effectively.  

Below we will create a file suitable for loading into cytoscape.  It will contain the three edge types of interest, linking drugs to genes, diseases to genes, and drugs to diseases e.g.:

source_node	source_type	edge_type	target_node	target_type


### Start with the drug and gene pairs

In [ ]:
drug_gene_links = simple_table[["drug", "gene"]]

In [ ]:
drug_gene_links.head()

,drug,gene
0,epinephrine,ADRA1D
1,norepinephrine,ADRA1D
2,clozapine,ADRA1D
3,terazosin,ADRA1D
4,indoramin,ADRA1D


### Rename the columns

In [ ]:
drug_gene_links = drug_gene_links.rename(columns = {"drug": "source_node", "gene": "target_node"})

In [ ]:
drug_gene_links.head()

,source_node,target_node
0,epinephrine,ADRA1D
1,norepinephrine,ADRA1D
2,clozapine,ADRA1D
3,terazosin,ADRA1D
4,indoramin,ADRA1D


### Create a new column specifying the source node type

In [ ]:
drug_gene_links["source_type"] = "drug"

In [ ]:
drug_gene_links.head()

,source_node,target_node,source_type
0,epinephrine,ADRA1D,drug
1,norepinephrine,ADRA1D,drug
2,clozapine,ADRA1D,drug
3,terazosin,ADRA1D,drug
4,indoramin,ADRA1D,drug


### Create a new column containing the edge type

In [ ]:
drug_gene_links["edge_type"] = "interacts_with"

In [ ]:
drug_gene_links.head()

,source_node,target_node,source_type,edge_type
0,epinephrine,ADRA1D,drug,interacts_with
1,norepinephrine,ADRA1D,drug,interacts_with
2,clozapine,ADRA1D,drug,interacts_with
3,terazosin,ADRA1D,drug,interacts_with
4,indoramin,ADRA1D,drug,interacts_with


### Create a new column containing the target type

In [ ]:
drug_gene_links["target_type"] = "gene"

In [ ]:
drug_gene_links.head()

,source_node,target_node,source_type,edge_type,target_type
0,epinephrine,ADRA1D,drug,interacts_with,gene
1,norepinephrine,ADRA1D,drug,interacts_with,gene
2,clozapine,ADRA1D,drug,interacts_with,gene
3,terazosin,ADRA1D,drug,interacts_with,gene
4,indoramin,ADRA1D,drug,interacts_with,gene


## Repeat for disease gene pairs

In [ ]:
disease_gene_links = simple_table[["disease", "gene"]]

In [ ]:
disease_gene_links.head()

,disease,gene
0,pre-eclampsia,ADRA1D
1,pre-eclampsia,ADRA1D
2,pre-eclampsia,ADRA1D
3,pre-eclampsia,ADRA1D
4,pre-eclampsia,ADRA1D


## Rename the columns

In [ ]:
disease_gene_links = disease_gene_links.rename(columns = {"disease": "source_node", "gene": "target_node"})

In [ ]:
disease_gene_links.head()

,source_node,target_node
0,pre-eclampsia,ADRA1D
1,pre-eclampsia,ADRA1D
2,pre-eclampsia,ADRA1D
3,pre-eclampsia,ADRA1D
4,pre-eclampsia,ADRA1D


### Create the new columns

In [ ]:
disease_gene_links["source_type"] = "disease"
disease_gene_links["edge_type"] = "associated_with"
disease_gene_links["target_type"] = "gene"

In [ ]:
disease_gene_links.head()

,source_node,target_node,source_type,edge_type,target_type
0,pre-eclampsia,ADRA1D,disease,associated_with,gene
1,pre-eclampsia,ADRA1D,disease,associated_with,gene
2,pre-eclampsia,ADRA1D,disease,associated_with,gene
3,pre-eclampsia,ADRA1D,disease,associated_with,gene
4,pre-eclampsia,ADRA1D,disease,associated_with,gene


In [ ]:
drug_disease_links = (simple_table
    [["drug", "disease"]]
    .assign(
        source_type = "drug",
        edge_type = "may treat",
        target_type = "disease"
    )
    .rename(columns = {"drug": "source_node", "disease": "target_node"})
)

# Join the (disease, gene), (drug, gene), and (drug, disease) tables together

### Number of rows of each table

In [ ]:
len(drug_gene_links)

1000

In [ ]:
len(disease_gene_links)

1000

In [ ]:
len(drug_disease_links)

1000

### Join all three tables together

In [ ]:
cytoscape_edges = pd.concat([drug_gene_links, disease_gene_links, drug_disease_links])

In [ ]:
cytoscape_edges.head()

,source_node,target_node,source_type,edge_type,target_type
0,epinephrine,ADRA1D,drug,interacts_with,gene
1,norepinephrine,ADRA1D,drug,interacts_with,gene
2,clozapine,ADRA1D,drug,interacts_with,gene
3,terazosin,ADRA1D,drug,interacts_with,gene
4,indoramin,ADRA1D,drug,interacts_with,gene


In [ ]:
len(cytoscape_edges)

3000

Note that the final result has 1785 (= 595 * 3) rows. (595 was the original number of results returned)

### Reorder columns

In [ ]:
cytoscape_edges = cytoscape_edges[["source_node", "source_type", "edge_type", "target_node", "target_type"]]

In [ ]:
cytoscape_edges.head()

,source_node,source_type,edge_type,target_node,target_type
0,epinephrine,drug,interacts_with,ADRA1D,gene
1,norepinephrine,drug,interacts_with,ADRA1D,gene
2,clozapine,drug,interacts_with,ADRA1D,gene
3,terazosin,drug,interacts_with,ADRA1D,gene
4,indoramin,drug,interacts_with,ADRA1D,gene


## Save file to disk

In [ ]:
cytoscape_edges.to_csv("drug_gene_disease_network.txt", sep = '\t', index = False, encoding = 'utf-8')

#open a new network in cytoscape and load the file by: File..Import..Network
#use Cytoscape to develop a visualization 
#submit the tab-delimited output files, an image of your network, and an explanation of what you did as your assignment
#Are there any notable hubs in the network?
#Could you extend the code to identify them automatically?
#can you adapt the code to search for a specific drug or a specific disease?